In [38]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression, Ridge, Lasso, RidgeCV, LassoCV, LogisticRegression
from sklearn.model_selection import cross_val_score,train_test_split, KFold, cross_val_predict
from sklearn.metrics import mean_squared_error,r2_score,roc_curve,auc,precision_recall_curve, accuracy_score, \
recall_score, precision_score, confusion_matrix
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold, RepeatedKFold, train_test_split
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,AdaBoostRegressor,AdaBoostClassifier, \
RandomForestRegressor
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
import itertools as it
import time as time
import re
from sklearn.preprocessing import StandardScaler
# from skopt import BayesSearchCV
# from skopt.space import Real, Categorical, Integer
# from skopt.plots import plot_objective, plot_histogram, plot_convergence
import warnings
from IPython import display

from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier

from xgboost import XGBClassifier, XGBRegressor
from sklearn.inspection import permutation_importance

In [6]:
train = pd.read_csv('train_regression.csv')
test = pd.read_csv('test_regression.csv')
train.head()

id    host_id  host_since host_location host_response_time  \
0            53872917  366516391  2018-04-09   Chicago, IL     within an hour   
1            39269663   50276775  2014-12-23           NaN     within an hour   
2  556480733030442463  681680651  2020-03-08           NaN     within an hour   
3            52397280   94345291  2015-10-22   Chicago, IL     within an hour   
4  745949202271858960  790998106  2021-04-04   Chicago, IL     within an hour   

  host_response_rate host_acceptance_rate host_is_superhost  \
0                96%                  95%                 f   
1               100%                  97%                 f   
2               100%                 100%                 f   
3               100%                  98%                 f   
4               100%                 100%                 t   

  host_neighbourhood  host_listings_count  host_total_listings_count  \
0  Back of the Yards                   11                         13   
1   Clearwater Beach                 1141                       2346   
2     Lake View East                    1                          1   
3           Bucktown                   55                         58   
4    Near South Side                   74                         75   

                 host_verifications host_has_profile_pic  \
0                ['email', 'phone']                    t   
1                ['email', 'phone']                    t   
2                         ['phone']                    t   
3  ['email', 'phone', 'work_email']                    t   
4                ['email', 'phone']                    t   

  host_identity_verified neighbourhood_cleansed   latitude  longitude  \
0                      t               New City  41.794240 -87.661380   
1                      t        Near North Side  41.892749 -87.620711   
2                      t              Lake View  41.948160 -87.652309   
3                      t              Hyde Park  41.801030 -87.597160   
4                      t        Near South Side  41.855770 -87.624130   

                 property_type        room_type  accommodates bathrooms_text  \
0  Private room in rental unit     Private room             1  1 shared bath   
1                Room in hotel     Private room            12        3 baths   
2           Entire rental unit  Entire home/apt             6         1 bath   
3           Entire rental unit  Entire home/apt             2         1 bath   
4           Entire rental unit  Entire home/apt             6        2 baths   

   beds      price  minimum_nights  maximum_nights  minimum_minimum_nights  \
0   1.0     $30.00              32            1125                      32   
1   3.0  $1,247.00              32             365                      32   
2   3.0    $214.00               2              45                       2   
3   1.0     $59.00               2             180                       2   
4   3.0    $151.00               2             365                       2   

   maximum_minimum_nights  minimum_maximum_nights  maximum_maximum_nights  \
0                      32                    1125                    1125   
1                      32                     365                     365   
2                       2                      45                      45   
3                       2                     180                     180   
4                       2                    1125                    1125   

   minimum_nights_avg_ntm  maximum_nights_avg_ntm has_availability  \
0                    32.0                  1125.0                t   
1                    32.0                   365.0                t   
2                     2.0                    45.0                t   
3                     2.0                   180.0                t   
4                     2.0                  1125.0                t   

   availability_30  availability_60  availability_90  availability_365  \
0           

In [7]:
pd.set_option('display.max_columns', None)

train['price'] = train['price'].str.replace('$', '').str.replace(',', '').astype(float)

train['host_response_rate'] = train['host_response_rate'].str.rstrip('%').astype(float)
train['host_acceptance_rate'] = train['host_acceptance_rate'].str.rstrip('%').astype(float)
train['host_since'] = pd.to_datetime(train['host_since'])

train['host_is_superhost'] = train['host_is_superhost'].apply(lambda x: 1 if x == 't' else 0)

train['instant_bookable'] = train['instant_bookable'].apply(lambda x: 1 if x == 't' else 0)
train['host_has_profile_pic'] = train['host_has_profile_pic'].apply(lambda x: 1 if x == 't' else 0)
train['host_identity_verified'] = train['host_identity_verified'].apply(lambda x: 1 if x == 't' else 0)
train['has_availability'] = train['has_availability'].apply(lambda x: 1 if x == 't' else 0)


'------------------------------------------------------------------------------------------------'

test['host_response_rate'] = test['host_response_rate'].str.rstrip('%').astype(float)
test['host_acceptance_rate'] = test['host_acceptance_rate'].str.rstrip('%').astype(float)
test['host_since'] = pd.to_datetime(test['host_since'])

test['instant_bookable'] = test['instant_bookable'].apply(lambda x: 1 if x == 't' else 0)
test['host_has_profile_pic'] = test['host_has_profile_pic'].apply(lambda x: 1 if x == 't' else 0)
test['host_identity_verified'] = test['host_identity_verified'].apply(lambda x: 1 if x == 't' else 0)
test['has_availability'] = test['has_availability'].apply(lambda x: 1 if x == 't' else 0)


# test['host_is_superhost'] = test['host_is_superhost'].apply(lambda x: 1 if x == 't' else 0)


# covert host_since into a usuable form
current_date = pd.to_datetime("today")

train['days_since_host_started'] = (current_date - train['host_since']).dt.days
test['days_since_host_started'] = (current_date - test['host_since']).dt.days

# Impute missing values

#Impute numerical columns with the mean
for column in train.select_dtypes(include=['number']).columns:
    train[column].fillna(train[column].mean(), inplace=True)
    
for column in test.select_dtypes(include=['number']).columns:
    test[column].fillna(train[column].mean(), inplace=True)  # Use train's mean to avoid data leakage

# Impute categorical columns with the mode (most frequent value)
for column in train.select_dtypes(include=['object', 'category']).columns:
    train[column].fillna(train[column].mode()[0], inplace=True)
    
for column in test.select_dtypes(include=['object', 'category']).columns:
    test[column].fillna(train[column].mode()[0], inplace=True)  # Use train's mode to avoid data leakage

# print(test.isnull().any())
# print('---------------------------------------------------------')
# print(train.isnull().any())

def extract_bathrooms(text):
    if 'half' in text.lower():
        return 0.5
    else:
        match = re.search(r'\d+\.?\d*', text)
        return float(match.group()) if match else np.nan

train['bathrooms'] = train['bathrooms_text'].apply(extract_bathrooms)
test['bathrooms'] = test['bathrooms_text'].apply(extract_bathrooms)

train['bathrooms'].fillna(train['bathrooms'].mean(), inplace=True)
test['bathrooms'].fillna(test['bathrooms'].mean(), inplace=True)

train.drop(columns=['bathrooms_text', 'host_id', 'host_since', 'first_review', 'last_review'], inplace=True)
test.drop(columns=['bathrooms_text', 'host_id', 'host_since', 'first_review', 'last_review'], inplace=True)

train = pd.get_dummies(train)
test = pd.get_dummies(test)

train.head()

#All columns in the dataset, except id, host_id, host_since, first_review, and last_review have
#to be tried as predictors. (price is the response, so it cannot be used as a predictor either.)

C:\Users\benja\AppData\Local\Temp\ipykernel_14616\2113949972.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  train['price'] = train['price'].str.replace('$', '').str.replace(',', '').astype(float)


id  host_response_rate  host_acceptance_rate  \
0            53872917                96.0                  95.0   
1            39269663               100.0                  97.0   
2  556480733030442463               100.0                 100.0   
3            52397280               100.0                  98.0   
4  745949202271858960               100.0                 100.0   

   host_is_superhost  host_listings_count  host_total_listings_count  \
0                  0                   11                         13   
1                  0                 1141                       2346   
2                  0                    1                          1   
3                  0                   55                         58   
4                  1                   74                         75   

   host_has_profile_pic  host_identity_verified   latitude  longitude  \
0                     1                       1  41.794240 -87.661380   
1                     1                       1  41.892749 -87.620711   
2                     1                       1  41.948160 -87.652309   
3                     1                       1  41.801030 -87.597160   
4                     1                       1  41.855770 -87.624130   

   accommodates  beds   price  minimum_nights  maximum_nights  \
0             1   1.0    30.0              32            1125   
1            12   3.0  1247.0              32             365   
2             6   3.0   214.0               2              45   
3             2   1.0    59.0               2             180   
4             6   3.0   151.0               2             365   

   minimum_minimum_nights  maximum_minimum_nights  minimum_maximum_nights  \
0                      32                      32                    1125   
1                      32                      32                     365   
2                       2                       2                      45   
3                       2                       2                     180   
4                       2                       2                    1125   

   maximum_maximum_nights  minimum_nights_avg_ntm  maximum_nights_avg_ntm  \
0                    1125                    32.0                  1125.0   
1                     365                    32.0                   365.0   
2                      45                     2.0                    45.0   
3                     180                     2.0                   180.0   
4                    1125                     2.0                  1125.0   

   has_availability  availability_30  availability_60  availability_90  \
0                 1               30               60               90   
1                 1               25               49               66   
2                 1                0                0                0   
3                 1               23               53               83   
4                 1                0                0                0   

   availability_365  number_of_reviews  number_of_reviews_ltm  \
0               365                 18                      0   
1               135                  0                      0   
2                 0                 14                     14   
3               355                 13                     13   
4                 0                 64                     42   

   number_of_reviews_l30d  review_scores_rating  review_scores_accuracy  \
0                       0              4.940000                 5.00000   
1                       0              4.759538                 4.79342   
2                       1              4.870000                 5.00000   
3                       3              4.080000                 4.08000   
4                       3              4.800000                 4.74000   

   review_scores_cleanliness  review_scores_checkin  \
0                   4.890000               5.000000   
1                   4.747335  

In [8]:
train_cols = list(train.columns)
test_cols = list(test.columns)

train_cols = [col for col in train_cols if col in test_cols]
test_cols = [col for col in test_cols if col in train_cols]

train_cols == test_cols

True

In [11]:
x_train = train[train_cols].drop(columns=['id'])
y_train = train.price

x_test = test[test_cols].drop(columns=['id'])

# Remove restricted characters
x_train.columns = x_train.columns.str.replace('[', '', regex=True).str.replace(']', '', regex=True).str.replace('<', '', regex=True)
x_test.columns = x_test.columns.str.replace('[', '', regex=True).str.replace(']', '', regex=True).str.replace('<', '', regex=True)

In [13]:
model = XGBRegressor(random_state = 1, 
                     objective='reg:squarederror',
                     
                     n_estimators=50, 
                     max_depth=6,
                     subsample=0.75,
                     learning_rate=0.1,
                    
                     reg_lambda = 1,
                     gamma = 0,
                    )
model.fit(x_train, y_train)

y_pred = model.predict(x_test)
   

In [19]:
# Narrowing features with lasso regression

x_train = train[train_cols].drop(columns=['id'])
y_train = train.price

x_test = test[test_cols].drop(columns=['id'])

sc = StandardScaler()

sc.fit(x_train)
x_train_scaled = sc.transform(x_train)
x_test_scaled = sc.transform(x_test)

lasso_model = Lasso(alpha=0.005, max_iter=400000)

x_train_subset = x_train_scaled[:500, :]

lasso_model.fit(x_train_subset, y_train[:500])

coefficients = pd.DataFrame({
    'feature': x_train.columns,
    'importance': np.abs(lasso_model.coef_)
})

coefficients.sort_values(by='importance', ascending=False, inplace=True)

important_features = list(coefficients.head(20).feature)

x_train_reduced = x_train[important_features]
x_test_reduced = x_test[important_features]

model = XGBRegressor(random_state = 1, 
                     objective='reg:squarederror',
                     
                     n_estimators=50, 
                     max_depth=6,
                     subsample=0.75,
                     learning_rate=0.1,
                    
                     reg_lambda = 1,
                     gamma = 0,
                    )

model.fit(x_train_reduced, y_train)

y_pred = model.predict(x_test_reduced)

In [21]:
# coarse grid

model = XGBRegressor(random_state=1)

grid = {
    'n_estimators' : [100, 500, 1000],
    'max_depth' : [4,6,8],
    'learning_rate' : [0.01, 0.1, 1],
    'subsample' : [0.5, 0.75, 1.0],
    'reg_lambda':[0.01, 0.1, 1],
    'gamma':[0, 0.1, 1] 
    
}

gscv = GridSearchCV(model, grid, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1, verbose=1).fit(x_train_reduced, y_train)

print(gscv.best_params_)
print(-gscv.best_score_)

y_pred = gscv.predict(x_test_reduced)

Fitting 5 folds for each of 729 candidates, totalling 3645 fits
{'gamma': 0, 'learning_rate': 0.01, 'max_depth': 8, 'n_estimators': 100, 'reg_lambda': 1, 'subsample': 0.5}
971.6679341088935


In [24]:
# finer grid

model = XGBRegressor(random_state=1)

grid = {
    'n_estimators' : [50, 100, 350],
    'max_depth' : [4,6,8],
    'learning_rate' : [0.001, 0.01, 0.02],
    'subsample' : [0.5, 0.6, 0.7],
    'reg_lambda':[0.5, 1, 2, 10],
    'gamma':[0]
    
}

gscv = GridSearchCV(model, grid, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1, verbose=1).fit(x_train_reduced, y_train)

print(gscv.best_params_)
print(-gscv.best_score_)

y_pred = gscv.predict(x_test_reduced)

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
{'gamma': 0, 'learning_rate': 0.01, 'max_depth': 8, 'n_estimators': 50, 'reg_lambda': 10, 'subsample': 0.6}
789.6164784357733


In [26]:
# finest grid

model = XGBRegressor(random_state=1)

grid = {
    'n_estimators' : [50, 100, 350],
    'max_depth' : [4,6,8],
    'learning_rate' : [0.005, 0.007, 0.01, 0.015],
    'subsample' : [0.57, 0.6, 0.63],
    'reg_lambda':[5, 10, 20, 100],
    'gamma':[0]
    
}

gscv = GridSearchCV(model, grid, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1, verbose=1).fit(x_train_reduced, y_train)

print(gscv.best_params_)
print(-gscv.best_score_)

y_pred = gscv.predict(x_test_reduced)

Fitting 5 folds for each of 432 candidates, totalling 2160 fits
{'gamma': 0, 'learning_rate': 0.01, 'max_depth': 8, 'n_estimators': 350, 'reg_lambda': 100, 'subsample': 0.57}
767.3469359155291


In [28]:
# cv settings, shuffle = true

cv_settings = KFold(n_splits=8, shuffle=True, random_state=1)

gscv = GridSearchCV(model, grid, cv=cv_settings, scoring='neg_root_mean_squared_error', n_jobs=-1, verbose=1).fit(x_train_reduced, y_train)

print(gscv.best_params_)
print(-gscv.best_score_)

y_pred = gscv.predict(x_test_reduced)

Fitting 8 folds for each of 432 candidates, totalling 3456 fits
{'gamma': 0, 'learning_rate': 0.01, 'max_depth': 8, 'n_estimators': 350, 'reg_lambda': 100, 'subsample': 0.57}
629.4233101390007


In [32]:
# cv settings, n_repeats = 5, n_splits =5

cv_settings = RepeatedKFold(n_splits=5, n_repeats=5, random_state=1)

gscv = GridSearchCV(model, grid, cv=cv_settings, scoring='neg_root_mean_squared_error', n_jobs=-1, verbose=1).fit(x_train_reduced, y_train)

print(gscv.best_params_)
print(-gscv.best_score_)

y_pred = gscv.predict(x_test_reduced)

Fitting 25 folds for each of 432 candidates, totalling 10800 fits
{'gamma': 0, 'learning_rate': 0.01, 'max_depth': 8, 'n_estimators': 350, 'reg_lambda': 100, 'subsample': 0.57}
754.0653122363105


In [34]:
# Narrow with decision tree importance
importances = np.array(gscv.best_estimator_.feature_importances_)
names = np.array(gscv.feature_names_in_)

important_features = pd.DataFrame({'feature':names, 'importance':importances})

important_features.sort_values(by='importance', inplace=True, ascending=False)



important_features = list(important_features.head(15).feature)

x_train_reduced = x_train[important_features]
x_test_reduced = x_test[important_features]

cv_settings = RepeatedKFold(n_splits=5, n_repeats=5, random_state=1)

gscv = GridSearchCV(model, grid, cv=cv_settings, scoring='neg_root_mean_squared_error', n_jobs=-1, verbose=1).fit(x_train_reduced, y_train)

print(gscv.best_params_)
print(-gscv.best_score_)

y_pred = gscv.predict(x_test_reduced)

Fitting 25 folds for each of 432 candidates, totalling 10800 fits
{'gamma': 0, 'learning_rate': 0.01, 'max_depth': 8, 'n_estimators': 350, 'reg_lambda': 100, 'subsample': 0.57}
753.2989699081937


In [39]:
# Narrowing features with permutation importance

model = gscv.best_estimator_
model.fit(x_train_reduced, y_train)

perm_importance = permutation_importance(model, x_train_reduced, y_train, n_repeats=10, random_state=1)

feature_names = [col for col in x_train_reduced if col != 'id' and col != 'price']
features_importance = pd.DataFrame({'feature': feature_names, 'importance': perm_importance.importances_mean})

features_importance.sort_values(by='importance', ascending=False, inplace=True)

important_features = list(features_importance.head(10).feature)

x_train_reduced = x_train[important_features]
x_test_reduced = x_test[important_features]

model = gscv.best_estimator_

model.fit(x_train_reduced, y_train)

y_pred = model.predict(x_test_reduced)

In [40]:
ids = test.id
results = pd.DataFrame({
    'id': ids,
    'predicted': y_pred
})


results.to_csv('XGboost_Regression_submission.csv', index=False)
results.head(30)

id   predicted
0   771986218856585018  204.363373
1   855276028675941785  139.889755
2             48537824  297.087372
3             41867473  144.109772
4             28361473  107.069366
5             51911862  111.023918
6   579514121718775234   97.149933
7             46242944   57.862190
8              5571262   89.976105
9   594937541144766707  136.583115
10  772410051557583562  127.879936
11            13165625   88.422501
12            29191606  165.533920
13            32381296  464.897156
14            50451083  156.552322
15            51565135  115.146309
16            47176352  123.649323
17  924507573112993030   69.032188
18            45653951  178.914490
19            26795754  114.010201
20            42035578   96.219070
21            49742536  127.848915
22            48932073  135.634018
23            15904833  118.579849
24  759540854095218350  105.694885
25            29885798  108.547630
26  655444310450112562  202.060760
27            11527162   83.453949
28            18735326  146.097946
29  606007670793701048  218.661774